In [ ]:
import os
import re
import json
from matplotlib import dates
import matplotlib.pyplot as plt
# Choose Style
# 'classic', 'ggplot', 'seaborn', 'Solarize_Light2', 'bmh', 'fivethirtyeight', 'dark_background', 'grayscale',  
plt.style.use('default')

from utils import *
from pprint import pprint

## Find files

In [ ]:
md_files = [
    '/home/wmnlab/D/database/2023-08-29/2023-08-29.md', 
    '/home/wmnlab/D/database/2023-09-12-1/2023-09-12-1.md',
    '/home/wmnlab/D/database/2023-09-22/2023-09-22.md',
    '/home/wmnlab/D/database/2023-10-24/2023-10-24.md',
    # '/home/wmnlab/D/database/2023-11-21/2023-11-21.md',
    # '/home/wmnlab/D/database/2023-12-26/2023-12-26.md',
    '/home/wmnlab/D/database/2024-03-10/2024-03-10.md',
    '/home/wmnlab/D/database/2024-03-16/2024-03-16.md',
    '/home/wmnlab/E/database/2024-04-18/2024-04-18.md',
    '/home/wmnlab/E/database/2024-05-08/2024-05-08.md'
    ]

EXPs = get_EXPs(md_files)
pprint(EXPs)

## Q1. Average $t_{cmd}$ to RRC reconnection